In [1]:
from pyquil.quil import Program
from pyquil.gates import *
import pyquil.api as api
import numpy as np

In [2]:
def phase_rotation(phi):
    '''
    Single-qubit phase rotation gate, parametrized by angle phi
    
    :param float phi: angle of phase rotation
    :return numpy.array: single-qubit gate representing phase rotation
    '''
    return np.array([[1, 0], [0, np.exp(1j*phi)]])

def controlled_phase_rotation(phi):
    '''
    Two-qubit controlled phase rotation gate, parametrized by angle phi
    
    :param float phi: angle of phase rotation
    :return numpy.array: two-qubit controlled phase rotation gate
    '''
    R_phi = phase_rotation(phi)
    proj_00 = np.array([[1, 0], [0, 0]])  # projection operator |0><0|
    proj_11 = np.array([[0, 0], [0, 1]])  # projection operator |1><1|
    # controlled phase shift of e^(i*phi) is symmetric with respect to swapping the control and target bits,
    # so can do in any order; here we'll keep the control qubits to the left
    return np.kron(proj_00, np.eye(2)) + np.kron(proj_11, R_phi)

def R_k(k):
    '''
    Controlled phase rotation, such that i*phi = (2*pi*i) / (2^k)
    
    :param int k: exponent of 2 in the denominator of the expression (2*pi*i) / (2^k)
    :return numpy.array: two-qubit controlled phase rotation gate, characterized by the integer k
    '''
    return controlled_phase_rotation((2*np.pi)/(2**k))

In [3]:
def state_prep(list_qubits, prog):
    '''
    Prepare the state corresponding to list of qubits, e.g. [1, 1, 0] -> |110>
    
    :param list list_qubits: list of qubits whose state is to be prepared
    :return Program: Program preparing the required qubit state
    '''
    n = len(list_qubits)
    for i, x in enumerate(list_qubits):
        q = n - 1 - i
        if x == 0:
            prog.inst(I(q))
        elif x == 1:
            prog.inst(X(q))
        else:
            raise ValueError("Input must consts of (integers) 0s and 1s")

def define_gates(n, prog):
    '''
    Define all the gates needed for an n-qubit QFT
    
    :param int n: number of qubits
    :param Program prog: Program
    '''
    for i in range(2, n+1):
        prog.defgate("R_" + str(i), R_k(i))

def qft(n, prog):
    '''
    Computes the Quantum Fourier Transform of an n-qubit state

    :param int n: number of qubits
    :return Program: Modified Program that would carry out quantum fourier transform of the input n-qubit state
    '''
    # define all the gates
    define_gates(n, prog)
    # create QFT circuit
    q = n - 1
    while q >= 0:
        # apply Hadamard
        prog.inst(H(q))
        for i in range(2, q+2):
#             # apply controlled phase rotations
#             print ("R_" + str(i))
#             print (tuple([q-i+1, q]))
#             print ("*" * 30)
            prog.inst(("R_" + str(i),) + tuple([q-i+1, q]))
        # decrement qubit index
        q -= 1
    # SWAP the order of qubits
    for j in range(n//2):
        prog.inst(SWAP(j, n-1-j))

In [4]:
qvm = api.QVMConnection()

# provide list of qubits
list_qubs = [0, 0, 1]
# prepare Program
p = Program()
state_prep(list_qubs, p)
qft(len(list_qubs), p)

# obtain wavefunction
wavefunc = qvm.wavefunction(p)
print (wavefunc.amplitudes)

ValueError: The wavefunction is not normalized. The probabilities sum to 0.4999999999999998 instead of 1

In [5]:
# print (wavefunc)
print (p)

DEFGATE R_2:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, 6.123233995736766e-17i

DEFGATE R_3:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, 0.7071067811865476+0.7071067811865475i

I 2
I 1
X 0
H 2
R_2 1 2
R_3 0 2
H 1
R_2 0 1
H 0
SWAP 0 2



In [51]:
cxn = api.QVMConnection()

# p1 = Program(I(2), I(1), X(0))
# # p1.defgate("R2", R_k(2))
# # p1.defgate("R3", R_k(3))
# p1.inst(H(2))
# # p1.inst(("R2", 1, 2))
# p1.inst(CPHASE(2*np.pi/(2**2), 1, 2))
# p1.inst(H(1))
# # p1.inst(("R3", 0, 2))
# p1.inst(CPHASE(2*np.pi/(2**3), 0, 2))
# # p1.inst(("R2", 0, 1))
# p1.inst(CPHASE(2*np.pi/(2**2), 0, 1))
# p1.inst(H(0))
# p1.inst(SWAP(0, 2))

p1 = Program(I(2), I(1), X(0))

p1.defgate("R2", R_k(2))
p1.defgate("R3", R_k(3))

p1.inst(H(2))
# p1.inst(CPHASE(2*np.pi/(2**2), 1, 2))
p1.inst(("R2", 2, 1))
# p1.inst(CPHASE(2*np.pi/(2**3), 0, 2))
p1.inst(("R3", 2, 0))
p1.inst(H(1))

# p1.inst(CPHASE(2*np.pi/(2**2), 1, 0))
p1.defgate("R2_2", R_k(2))
p1.inst(("R2_2", 1, 0))
# p1.inst(("R2", 1, 0))

# p1.inst(H(0))
# p1.inst(SWAP(0, 2))

print (p1)

wavefunc1 = cxn.wavefunction(p1)
print (wavefunc1)
print ("*" * 30)
# print (np.allclose(wavefunc1.amplitudes, np.array([  3.53553391e-01+0.j        ,   2.50000000e-01+0.25j      ,
#          2.16489014e-17+0.35355339j,  -2.50000000e-01+0.25j      ,
#         -3.53553391e-01+0.j        ,  -2.50000000e-01-0.25j      ,
#         -2.16489014e-17-0.35355339j,   2.50000000e-01-0.25j      ])))

DEFGATE R2:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, 6.123233995736766e-17i

DEFGATE R3:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, 0.7071067811865476+0.7071067811865475i

DEFGATE R2_2:
    1.0, 0.0, 0.0, 0.0
    0.0, 1.0, 0.0, 0.0
    0.0, 0.0, 1.0, 0.0
    0.0, 0.0, 0.0, 6.123233995736766e-17i

I 2
I 1
X 0
H 2
R2 2 1
R3 2 0
H 1
R2_2 1 0



ValueError: The wavefunction is not normalized. The probabilities sum to 0.4999999999999998 instead of 1

In [52]:
R_k(2)

array([[1.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 1.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 1.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17+1.j]])

In [44]:
(0.3535533906**2) - (1/8.)

4.756167681918555e-12

In [9]:
cxn = api.QVMConnection()

p1 = Program(I(2), I(1), X(0))
p1.defgate("R", R_k(2))
p1.inst(("R", 1, 2))
wavefunc1 = cxn.wavefunction(p1)
print (wavefunc1)

p2 = Program(I(2), I(1), X(0))
p2.inst(CPHASE((2*np.pi)/(2**2), 1, 2))
wavefunc2 = cxn.wavefunction(p2)
print (wavefunc2)

(1+0j)|001>
(1+0j)|001>


In [ ]:
# list_qubits = [1, 1, 1]
# qub_vec = [0 for _ in range(2**(len(list_qubits)))]
qub_vec = [0 for _ in range(2**(len(list_qubs)))]
lexicographic_index = 0
for i, x in enumerate(list_qubits[::-1]):
    lexicographic_index += x * (2**i)
qub_vec[lexicographic_index] = 1
        
print (qub_vec)

In [ ]:
from numpy.fft import ifft

ifft(qub_vec, norm="ortho")